In [10]:
import os
for k, v in dict(
    EMAIL_TO='agalea91@gmail.com',
    EMAIL_FROM='Toc <toc.adi.now@gmail.com>',
    NOTION_DATABASE_ID='',
    NOTION_SECRET_TOKEN='',
    SENDGRID_SECRET_TOKEN=''
).items():
    os.environ[k] = v

In [48]:
import requests
import os

HEADERS = {
    "Authorization": f"Bearer {os.getenv('NOTION_SECRET_TOKEN')}",
    "Notion-version": "2021-05-13",
}


def retrieve_database(database_id, query=None, start_cursor=None) -> dict:
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    data = query or {}
    if start_cursor:
        data["start_cursor"] = start_cursor
    print(f"POST {url}")
    response = requests.post(url=url, headers=HEADERS, json=data)
    response.raise_for_status()
    database = response.json()
    return database




def retrieve_page_content(page_id) -> dict:
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    print(f"GET {url}")
    response = requests.get(url=url, headers=HEADERS)
    response.raise_for_status()
    page = response.json()
    return page

In [49]:
import re
import random
import json


def get_random_page(database):
    if database["has_more"]:
        raise NotImplementedError("Database has more pages. Must implement pagination.")

    idx = random.choice(list(range(len(database["results"]))))
    page = database["results"][idx]
    return page


def get_page_id(database_page):
    return re.findall(r"/([^/]+)$", database_page["url"])[0]


def get_page_topic(database_page):
    return database_page["properties"]["Tags"]["multi_select"][0]["name"]


def parse_page_quote_content(block_page):
    try:
        quote = block_page["results"][0]["paragraph"]["text"][0]["text"]["content"]
        attrib = block_page["results"][1]["paragraph"]["text"][0]["text"]["content"]
    except Exception as e:
        print(f"Error parsing quote from page {json.dumps(block_page, indent=2)}")
        raise
    return quote, attrib

In [23]:

database1 = retrieve_database(
    database_id=os.getenv("NOTION_DATABASE_ID"),
    start_cursor=None
)

POST https://api.notion.com/v1/databases/bb7a2ecc460f43dbaf751cf1ee6d27ba/query


In [24]:
database1["has_more"]

True

In [43]:
database1["next_cursor"]

'59a3f846-0db7-48f3-89f9-d5ffd110922f'

In [50]:
database2 = retrieve_database(
    database_id=os.getenv("NOTION_DATABASE_ID"),
    start_cursor=database1["next_cursor"]
)

POST https://api.notion.com/v1/databases/bb7a2ecc460f43dbaf751cf1ee6d27ba/query


In [51]:
database2["has_more"]

False

In [52]:
len(database1["results"])

100

In [53]:

len(database2["results"])

6

In [54]:
database1["results"][0]["url"]

'https://www.notion.so/f04302363a184525b04e142cc3c1f1aa'

In [55]:

database2["results"][0]["url"]

'https://www.notion.so/59a3f8460db748f389f9d5ffd110922f'

In [58]:
type(database1)

dict